In [ ]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import numpy as np
import os

# Solicitar el archivo txt al usuario
from google.colab import files

print("Por favor, sube el archivo .txt con las intervenciones del parlamentario.")
uploaded = files.upload()

# Leer el archivo .txt
filename = list(uploaded.keys())[0]
with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocesamiento del texto
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = text.replace('\n', ' ')  # Reemplazar saltos de línea por espacios
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    return text

text = preprocess_text(text)
words = text.split()  # Dividir el texto en palabras únicas
vocab = sorted(set(words))  # Crear vocabulario único

# Crear diccionarios de mapeo
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# Convertir el texto a índices
encoded_text = [word_to_idx[word] for word in words]

# Preparar los datos para el modelo
sequence_length = 20  # Aumentar la longitud de las secuencias
X = []
y = []

for i in range(len(encoded_text) - sequence_length):
    X.append(encoded_text[i:i+sequence_length])
    y.append(encoded_text[i+sequence_length])

X = np.array(X)
y = np.array(y)

# Construir un modelo LSTM mucho más potente
vocab_size = len(vocab)
embedding_dim = 256  # Aumentar la dimensión de los embeddings
lstm_units = 1024  # Aumentar el número de unidades en LSTM

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=sequence_length),
    LSTM(lstm_units, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.3),  # Mayor dropout para evitar overfitting
    LSTM(lstm_units, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.3),
    GRU(lstm_units, return_sequences=False, kernel_regularizer=regularizers.l2(0.01)),  # Añadir una capa GRU
    Dropout(0.3),
    Dense(vocab_size, activation='softmax')
])

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Añadir callback para guardar el modelo con el mejor rendimiento
checkpoint = ModelCheckpoint('best_model.keras', monitor='loss', save_best_only=True)

# Entrenamiento largo, con más épocas y callback
print("Entrenando el modelo... Esto puede tomar varias horas.")
model.fit(X, y, epochs=150, batch_size=256, verbose=1, callbacks=[checkpoint])

# Función para generar texto con temperatura
def generate_text(model, start_sequence, max_words=500, temperature=1.0):
    input_sequence = [word_to_idx[word] for word in start_sequence.split() if word in word_to_idx]
    generated = start_sequence

    for _ in range(max_words):
        input_padded = tf.keras.preprocessing.sequence.pad_sequences([input_sequence], maxlen=sequence_length)
        predictions = model.predict(input_padded, verbose=0).flatten()
        predictions = np.log(predictions + 1e-9) / temperature
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        predicted_idx = np.random.choice(len(probabilities), p=probabilities)
        predicted_word = idx_to_word[predicted_idx]

        generated += ' ' + predicted_word
        input_sequence.append(predicted_idx)
        input_sequence = input_sequence[1:]

        # Finalizar si se alcanza un final lógico
        if predicted_word in ['.', '!', '?']:
            break

    return generated

# Interactuar con el modelo sin preguntar la longitud
print("¡Puedes comenzar a interactuar con el modelo!")
while True:
    prompt = input("Introduce el inicio del texto (o escribe 'salir' para terminar): ")
    if prompt.lower() == 'salir':
        print("¡Hasta luego!")
        break
    print("Respondiendo...")
    print(generate_text(model, prompt, max_words=500, temperature=0.8))


Por favor, sube el archivo .txt con las intervenciones del parlamentario.


Saving intervencionesCasado.txt to intervencionesCasado (1).txt
Entrenando el modelo... Esto puede tomar varias horas.
Epoch 1/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.0365 - loss: 20.7967
Epoch 2/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - accuracy: 0.0465 - loss: 6.3152
Epoch 3/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - accuracy: 0.0452 - loss: 6.2540
Epoch 4/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - accuracy: 0.0469 - loss: 6.2870
Epoch 5/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - accuracy: 0.0467 - loss: 6.2470
Epoch 6/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.0467 - loss: 6.2476
Epoch 7/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - accuracy: 0.0458 - loss: 6.2526
Epoch 8/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.0470 - loss: 6.2486
Epoch 9/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - accuracy: 0.0487 - loss: 6.2360
Epoch 10/150
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - accuracy: 0.0469 - loss: 6.2525
Epoch 11/150
72/72